# Introduction

"""
@author: ArturKovalchuk
"""

**
IMPORTANT:
works only with Python 3.5**
1. Install Magic Wand (need older version)
available here: https://ftp.icm.edu.pl/packages/ImageMagick/binaries/
    ImageMagick-6.9.8-9-Q16-x64-dll.exe
2. Do this:
    http://docs.wand-py.org/en/latest/guide/install.html#install-imagemagick-on-windows
or here: http://docs.wand-py.org/en/0.2-maintenance/_images/windows-envvar.png
3. From Conda install Wand:
    pip install Wand
4. https://github.com/UB-Mannheim/tesseract/wiki    and here    https://github.com/UB-Mannheim/tesseract/wiki

https://ghostscript.com/Ghostscript_9.22.html

Additional info:

https://gist.github.com/dannguyen/a0b69c84ebc00c54c94d

https://testpypi.python.org/pypi/pytesseract

https://github.com/tesseract-ocr/tesseract/wiki

https://github.com/UB-Mannheim/tesseract/wiki


In order to create new environment, You need to run these commands in the console:
- **conda create -n python_3.5 python=3.5 anaconda** # creates new environment with Python 3.5
- **pip install wand** # installs wand for converting pdf to jpg
- **pip install pytesseract** # installs pytesseract for OCR

**Activate Google Vision API, including credentials**

### Check the environment we are working in:

In [1]:
#Check the environment python version
import sys
print(sys.version)

3.5.4 |Anaconda custom (64-bit)| (default, Sep 19 2017, 08:15:17) [MSC v.1900 64 bit (AMD64)]


### Import the liabraries we need:

In [2]:
# for working with files    
import io
import os

# for data manipulation
import pandas as pd

# for downloading
import requests

# for converting pdf to jpg with processing
from wand.image import Image as wImage

# Imports the Google Cloud client library
from google.cloud import vision
from google.cloud.vision import types    

# IMPORTANT: activate the credentials for Google API
# Instantiates a client
client = vision.ImageAnnotatorClient()#creds=json.load(open('Prozorro_prices-febfc26c2945.json')) #credentials=creds

### Create the functions we need:

In [ ]:
def download_contract(url, contract_id):
    """The unction takes contract's url and id, and downloads conttract to the disc"""
    response = requests.get(url)

    file_name =contract_id + ".pdf"
    with open(file_name, 'wb') as f:
         f.write(response.content)
    return file_name

In [ ]:
def convert_pdf_to_single_page_img(file_name, start_page=0, end_page=0):  
    """Function takes pdf file and converts it to the single-page image.
        Input:
            file_name - name of the pdf-file;
            start_page - starting page of the pdf-file to convert;
            end_page - final page of the pdf-file to convert.
        Output:
            name of the single-page image file saved.
    """
    with wImage(filename=file_name, resolution=300) as img:
        width_pdf=max([image.width for image in img.sequence]) # width of the document
        height_pdf=sum([image.height for image in img.sequence]) # lenght of the document
        
        with wImage() as blankimage:
            blankimage.blank(width=width_pdf, height=height_pdf)
            
            width_blank=0; height_blank=0
            for page_num in range(start_page, len(img.sequence)-end_page):
                img_to_save=wImage(img.sequence[page_num])
                img_to_save.threshold(0.7)
                img_to_save.compression_quality = 100
                img_to_save.format='png'
                
                blankimage.composite(img_to_save, width_blank, height_blank)
                height_blank+=img_to_save.height
            
            blankimage.save(filename = file_name+'.png')
        
    return file_name+'.png'

In [5]:
# function to iterate through lines
def lines_iteration(text): 
    return iter(text.splitlines())

### Code

Create the list of contracts to read:

In [4]:
# contract_id_list=["08a5089c916949cbba4f95bc4caddb8d"]
# #contract_name_list=["Овочі Єкименко.pdf"]
# contract_url_list=["https://public.docs.openprocurement.org/get/6c04c481772947108617d78c14151814?KeyID=52462340&Signature=z5IrOBxwdbra7ma56JmVFmLu6GA8Qcffu%252BIEId%2FUmCVGmZJjYgSW2FTema6kMjcMWc2ZRjD45Apj%252BIscAX0wBw%253D%253D"]
# contracts_id_url=dict(zip(contract_id_list, contract_url_list))

In [6]:
contracts=pd.read_excel("test_2.xlsx")
contracts.columns=["contract_id", "contract_url", "contract_name", "contract_format"]

contract_id_list=list(contracts["contract_id"][contracts["contract_format"]=="application/pdf"])[:10]
contract_url_list=list(contracts["contract_url"][contracts["contract_format"]=="application/pdf"])[:10]

contracts_id_url=dict(zip(contract_id_list, contract_url_list))

Download and convert the text:

# Append

In [27]:
# file_name=download_contract(contract_url_list[0], contract_id_list[0])                  
# picture_names=convert_pdf_to_img(file_name, last_page=True)
# degree=image_rotation(picture_names[0])

In [25]:
file_name="defa22a09d0346399c6246a3c74858a8.pdf"

In [26]:
convert_pdf_to_single_page_img(file_name, start_page=0, end_page=0)

'defa22a09d0346399c6246a3c74858a8.pdf.png'

In [ ]:
table_raw=[]

list_of_goods=["a-95", "дизельне паливо", "a-92"] # list of goods we are looking for

for contract_id, contract_url in contracts_id_url.items(): # iterate over contracts
    file_name=download_contract(contract_url, contract_id) # download the files, returns the file's name
    picture_names=convert_pdf_to_img(file_name, last_page=True) # converts downloaded file into images
    for picture in picture_names: # iterate over images of the contract
        text_raw=pytesseract.image_to_string(Image.open(picture), lang="ukr").lower() # convert image into the text
        try:
            text_raw=text_raw.split("специфікація")[1]
        except IndexError: # break if there is no specification
            break
        text_raw_list=list(lines_iteration(text_raw)) # converts text into the list of rows
        print("Picture's name: ", picture)
        print(text_raw_list)
        print("________________")
        print("________________")
        text_w_goods_list=[]
        for good in list_of_goods:
            for line in text_raw_list:
                if good in line:
                    text_w_goods_list.append([line])
    table_raw.append(text_w_goods_list)
table_raw